# Getting Data from the user: 

In [ ]:
#@title Upload Historical Data:

#@markdown ---

from google.colab import files

uploaded = files.upload()

file_name = list(uploaded.keys())[0]

Saving Student_Performance_Data.xlsx to Student_Performance_Data.xlsx


In [ ]:
#@title Select the Semester and Paper(s): { run: "auto" }

#@markdown ### Select Semester:

Semester = 'Sem_6' #@param ["Sem_1", "Sem_2", "Sem_3", "Sem_4", "Sem_5", "Sem_6", "Sem_7", "Sem_8"]
key = Semester

known_solution = []

#@markdown ### Select Paper:
Paper_1 = False #@param {type:"boolean"}
Paper_2 = False #@param {type:"boolean"}
Paper_3 = True #@param {type:"boolean"}
Paper_4 = True #@param {type:"boolean"}
Paper_5 = False #@param {type:"boolean"}
Paper_6 = False #@param {type:"boolean"}
Paper_7 = False #@param {type:"boolean"}

if Paper_1:
    known_solution.append('Paper 1')

if Paper_2:
    known_solution.append('Paper 2')

if Paper_3:
    known_solution.append('Paper 3')

if Paper_4:
    known_solution.append('Paper 4')

if Paper_5:
    known_solution.append('Paper 5')

if Paper_6:
    known_solution.append('Paper 6')

if Paper_7:
    known_solution.append('Paper 7')

#@markdown ### Enter Marks Threshold:

marks_thresh = 75 #@param {type:"slider", min:0, max:100, step:1}

#@markdown ### Number of Top Recommendations For Calculation:

Number_of_top_recommendations = 4 #@param {type:"slider", min:2, max:7, step:1}
n_similarity_values = Number_of_top_recommendations

#@markdown ### Actual Number of Recommendations required:
#@markdown ###### Note: This Value must be <= Number_of_top_recommendations

req_number_of_recommendations = 2 #@param {type:"slider", min:2, max:7, step:1}

n_recommendation_sem = req_number_of_recommendations



# Code:

## Importing Libraries:

In [ ]:
import pandas as pd
import numpy as np
from scipy import sparse
from sklearn.metrics.pairwise import cosine_similarity

## Preparing the Data:

In [ ]:
### Input and get the semester:
df = pd.read_excel(file_name, sheet_name='Sheet1')

df_dict_sem = {}
for name in df['Semster_Name'].unique():
    df_dict_sem[name] = df[df['Semster_Name'] == name].reset_index(drop=True)

df_sem = df_dict_sem[key]

In [ ]:
#Applying Marks Threshold
df_sem_marks_thresh = df_sem[df_sem['Marks'] > marks_thresh].reset_index(drop=True)
df_sem_marks_thresh.head()

,Student_ID,Semster_Name,Paper_ID,Paper_Name,Marks
0,SID20131143,Sem_6,SEMI0068923,Paper 1,98
1,SID20131143,Sem_6,SEMI0067259,Paper 2,84
2,SID20131143,Sem_6,SEMI0064181,Paper 4,76
3,SID20131151,Sem_6,SEMI0065623,Paper 3,87
4,SID20131151,Sem_6,SEMI0064181,Paper 4,92


In [ ]:
#getting only the sudent ID and the paper name
df_sem_marks_thresh_ID_Paper = df_sem_marks_thresh[['Student_ID', 'Paper_Name']]
df_sem_marks_thresh_ID_Paper

,Student_ID,Paper_Name
0,SID20131143,Paper 1
1,SID20131143,Paper 2
2,SID20131143,Paper 4
3,SID20131151,Paper 3
4,SID20131151,Paper 4
...,...,...
10580,SID20189982,Paper 2
10581,SID20189982,Paper 3
10582,SID20189989,Paper 5
10583,SID20189989,Paper 6


In [ ]:
#encoding
df_sem_dummies = pd.get_dummies(df_sem_marks_thresh_ID_Paper, prefix=[None], columns=['Paper_Name'])
df_sem_encoded = df_sem_dummies.groupby(['Student_ID'], axis=0).sum().reset_index()
df_sem_encoded.head()

,Student_ID,Paper 1,Paper 2,Paper 3,Paper 4,Paper 5,Paper 6,Paper 7
0,SID20131143,1,1,0,1,0,0,0
1,SID20131151,0,0,1,1,0,0,0
2,SID20131171,0,0,0,0,0,0,1
3,SID20131176,0,0,0,1,1,1,1
4,SID20131177,1,0,1,0,0,0,1


In [ ]:
#Adding Input as an entry to the data
ser = pd.Series(0, index = df_sem_encoded.columns)
ser[known_solution] = 1
ser['Student_ID'] = 'NEW_VALUE'
ser

df_sem_encoded = df_sem_encoded.append(ser, ignore_index=True)
df_sem_encoded.head()

,Student_ID,Paper 1,Paper 2,Paper 3,Paper 4,Paper 5,Paper 6,Paper 7
0,SID20131143,1,1,0,1,0,0,0
1,SID20131151,0,0,1,1,0,0,0
2,SID20131171,0,0,0,0,0,0,1
3,SID20131176,0,0,0,1,1,1,1
4,SID20131177,1,0,1,0,0,0,1


## Item-Item Calculations:

In [ ]:
#dropping student ID column for calculation purposes
data_sem_calc = df_sem_encoded.drop(columns="Student_ID")
data_sem_calc.head()

,Paper 1,Paper 2,Paper 3,Paper 4,Paper 5,Paper 6,Paper 7
0,1,1,0,1,0,0,0
1,0,0,1,1,0,0,0
2,0,0,0,0,0,0,1
3,0,0,0,1,1,1,1
4,1,0,1,0,0,0,1


In [ ]:
#normalizing
magnitude = np.sqrt(np.square(data_sem_calc).sum(axis=1))
data_sem_calc = data_sem_calc.divide(magnitude, axis='index')

In [ ]:
#Taking Age into account (To be done later)

In [ ]:
#Calculating Similartiy of items and store it in a sparse matrix
def calculate_similarity(data_sem_calc):
    data_sparse = sparse.csr_matrix(data_sem_calc)
    similarities =  cosine_similarity(data_sparse.transpose())
    df_sim = pd.DataFrame(data=similarities, index=data_sem_calc.columns, columns=data_sem_calc.columns)
    return df_sim

data_sem_matrix = calculate_similarity(data_sem_calc)
data_sem_matrix

,Paper 1,Paper 2,Paper 3,Paper 4,Paper 5,Paper 6,Paper 7
Paper 1,1.000000,0.319898,0.310327,0.304415,0.317802,0.302104,0.315793
Paper 2,0.319898,1.000000,0.307909,0.305486,0.313955,0.305361,0.308965
Paper 3,0.310327,0.307909,1.000000,0.297598,0.305426,0.303543,0.312575
Paper 4,0.304415,0.305486,0.297598,1.000000,0.298489,0.291117,0.308194
Paper 5,0.317802,0.313955,0.305426,0.298489,1.000000,0.313718,0.327550
Paper 6,0.302104,0.305361,0.303543,0.291117,0.313718,1.000000,0.318071
Paper 7,0.315793,0.308965,0.312575,0.308194,0.327550,0.318071,1.000000


## User-Item Calculations:

In [ ]:
#Storing top 4 papers 
#n_similarity_values = 4
data_sem_neighbours = pd.DataFrame(index=data_sem_matrix.columns, columns = range(1, n_similarity_values+1))
for i in range(len(data_sem_matrix.columns)):
    data_sem_neighbours.iloc[i, :n_similarity_values] = data_sem_matrix.iloc[0:, i].sort_values(ascending=False)[:n_similarity_values].index
data_sem_neighbours

,1,2,3,4
Paper 1,Paper 1,Paper 2,Paper 5,Paper 7
Paper 2,Paper 2,Paper 1,Paper 5,Paper 7
Paper 3,Paper 3,Paper 7,Paper 1,Paper 2
Paper 4,Paper 4,Paper 7,Paper 2,Paper 1
Paper 5,Paper 5,Paper 7,Paper 1,Paper 2
Paper 6,Paper 6,Paper 7,Paper 5,Paper 2
Paper 7,Paper 7,Paper 5,Paper 6,Paper 1


In [ ]:
#List of papers most similar to known_solution
most_similar_to_solution_sem = data_sem_neighbours.loc[known_solution]
similar_list_sem = most_similar_to_solution_sem.values.tolist()
similar_list_sem = list(set([item for sublist in similar_list_sem for item in sublist]))
similar_list_sem

['Paper 3', 'Paper 4', 'Paper 1', 'Paper 7', 'Paper 2']

In [ ]:
#similarity b/w above mentioned similar elements
neighbourhood_sem = data_sem_matrix[similar_list_sem].loc[similar_list_sem]
neighbourhood_sem

,Paper 3,Paper 4,Paper 1,Paper 7,Paper 2
Paper 3,1.000000,0.297598,0.310327,0.312575,0.307909
Paper 4,0.297598,1.000000,0.304415,0.308194,0.305486
Paper 1,0.310327,0.304415,1.000000,0.315793,0.319898
Paper 7,0.312575,0.308194,0.315793,1.000000,0.308965
Paper 2,0.307909,0.305486,0.319898,0.308965,1.000000


In [ ]:
#possible solutions that exist based on similarity List
Solution_index_sem = df_sem_encoded[df_sem_encoded.Student_ID == 'NEW_VALUE'].index.tolist()[0]
Solution_vector_sem = data_sem_calc.iloc[Solution_index_sem].loc[similar_list_sem]
Solution_vector_sem

Paper 3    0.707107
Paper 4    0.707107
Paper 1    0.000000
Paper 7    0.000000
Paper 2    0.000000
Name: 3709, dtype: float64

In [ ]:
#calculating score for final result (Cosine similarity)
score_sem = neighbourhood_sem.dot(Solution_vector_sem).div(neighbourhood_sem.sum(axis=1))
score = score_sem.drop(known_solution)
score

Paper 1    0.193158
Paper 7    0.195477
Paper 2    0.193437
dtype: float64

# Final Output:


## Known: 

In [ ]:
print('Known Solution: ', known_solution)

Known Solution:  ['Paper 3', 'Paper 4']


## Recommended: 

In [ ]:
#n_recommendation_sem = 2
top_n_recommendation = score.nlargest(n_recommendation_sem).index.tolist()
print('Top Recommendations:', top_n_recommendation)
print('Top Scores')
print(score)

Top Recommendations: ['Paper 7', 'Paper 2']
Top Scores
Paper 1    0.193158
Paper 7    0.195477
Paper 2    0.193437
dtype: float64
